<a href="https://colab.research.google.com/github/gagoorisamchon/aalto/blob/main/aifinance_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>